In [1]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [247]:
def sort_by_official_dates(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date <= date1:
                df1 = df1.append(row, ignore_index=True)
            if date > date1 and date <= date2:
                df2 = df2.append(row, ignore_index=True)
            if date > date2 and date <= date3:
                df3 = df3.append(row, ignore_index=True)
            if date > date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def sort_by_unofficial_dates(df):
    df1 = pd.DataFrame() # < 2004
    df2 = pd.DataFrame() # 2004 - 2008
    df3 = pd.DataFrame() # 2008 - 2014
    df4 = pd.DataFrame() # > 2014
    date1 = datetime.strptime('21-11-2004', '%d-%m-%Y').date()
    date2 = datetime.strptime('01-01-2008', '%d-%m-%Y').date()
    date3 = datetime.strptime('01-01-2014', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date < date1:
                df1 = df1.append(row, ignore_index=True)
            if date >= date1 and date < date2:
                df2 = df2.append(row, ignore_index=True)
            if date >= date2 and date < date3:
                df3 = df3.append(row, ignore_index=True)
            if date >= date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def get_stat(dfs):
    n = 0
    for df in dfs:
        print('Corpus {}:'.format(n), len(df), 'texts', len(get_all_texts(df).split()), 'words')
        n += 1

def show_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}:'.format(topic_idx))
        print(' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(documents, n_topics, n_top_words, models):
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        show_topics(nmf, tfidf_feature_names, n_top_words)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        show_topics(lda, tf_feature_names, n_top_words)

### Sorting data by official dates

In [3]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_official_dates(df)

In [170]:
# statistics 
get_stat(sorted_dfs)

Corpus 0: 1583 texts 845988 words
Corpus 1: 3168 texts 1111520 words
Corpus 2: 3344 texts 1082001 words
Corpus 3: 4277 texts 1420393 words


In [4]:
# get unnecessary symbols to remove them from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
print(symbols_to_del)

['"', '´', '8', '4', '`', '&', '−', '=', '„', '%', '！', ']', '?', '－', '$', '3', '1', '0', '…', '\xad', '€', '+', ',', '.', '@', '№', '’', ':', '―', '‒', '>', '\n', '«', '*', '#', '(', '·', '“', '”', '[', '_', '/', '5', '2', '‑', '6', '{', '–', '»', ')', '<', '9', '́', '°', '!', ';', '—', '7', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
# clean texts from unnecessary symbols

for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [208]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [lemmatize_pymorphy(word) for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

stopwords.extend('''б г д е ж з й л м н п р т ф х ц ч ш щ ъ ы ь э ю
                наш твой мой свой ваш его их ее над чей такой другой между год страна государство также все отношение г-н 
                россия российский сторона де два дж мид внешнеполитический интервью министр газета иностранный дело
                катерина спецкор новость рассказывать подобный кроме министерство кстати пресс-конференция москва сайт лавров
                медведев путин сайт подробный рф s форум сердечно добрый желать приветствовать сам ясный ясность кто-то
                говорить хотеть официальный еще какой-то об лишь включая вокруг ле никакой во'''.split())

print(stopwords)

['то', 'она', 'мы', 'из', 'что', 'бы', 'и', 'себя', 'чтобы', 'очень', 'но', 'быть', 'как', 'весь', 'вот', 'ты', 'по', 'здесь', 'потому', 'это', 'за', 'можно', 'а', 'ничто', 'этот', 'или', 'мочь', 'под', 'потом', 'до', 'нет', 'при', 'человек', 'они', 'вы', 'же', 'сказать', 'есть', 'не', 'с', 'где', 'только', 'о', 'ну', 'я', 'который', 'через', 'ни', 'более', 'от', 'он', 'уже', 'после', 'для', 'там', 'в', 'на', 'у', 'без', 'теперь', 'кто', 'так', 'раз', 'тот', 'ли', 'к', 'когда', 'если', 'время', 'тем', 'чем', 'надо', 'один', 'даже', 'ещё', 'тут', 'тогда', 'б', 'г', 'д', 'е', 'ж', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'такой', 'другой', 'между', 'год', 'страна', 'государство', 'также', 'все', 'отношение', 'г-н', 'россия', 'российский', 'сторона', 'де', 'два', 'дж', 'мид', 'внешнеполитический', 'интервью', 'министр', 'газета', 'иностранный', 'дело', 'катерина', 'с

In [209]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b['\w-]+\b")
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b['\w-]+\b")

In [238]:
get_topics(sorted_dfs[0].loc[:, 'text_clean'], 33, 15, ['nmf']) # до 2008

NMF
Topic 0:
международный развитие новый политика мир мировой интерес проблема становиться решение работа глобальный важный безопасность экономический
Topic 1:
израиль палестинец дорожный карта израильский палестино-израильский палестинский четверка посредник квартет урегулирование пна израильтянин ближневосточный аббас
Topic 2:
сотрудничество вопрос обсуждать переговоры визит президент договариваться двусторонний встреча договоренность взаимодействие сегодня сфера торговый-экономический коллега
Topic 3:
палестинский пна аббас хамас администрация территория межпалестинский глава правительство национальный народ израиль обстановка мирный палестинец
Topic 4:
европа совет се европейский председательство конвенция комитет парламентский правовой общеевропейский право ассамблея разделительный организация демократия
Topic 5:
исламский цивилизация оик ислам религия религиозный православный мусульманский мир церковь культура мусульманин межцивилизационный конференция диалог
Topic 6:
соотечеств

In [162]:
get_topics(sorted_dfs[1].loc[:, 'text_clean'], 39, 15, ['nmf']) # 2008 - 2013

NMF
Topic 0:
международный мир развитие новый глобальный мировой становиться работа безопасность интерес основа политика решение система проблема
Topic 1:
осетия южный абхазия республика закавказье женевский граница абхазский грузия неприменение дискуссия югоосетинский сила соглашение цхинвал
Topic 2:
сотрудничество двусторонний взаимодействие обсуждать переговоры вопрос сфера область коллега торговый-экономический связь совместный ход развитие сегодня
Topic 3:
соотечественник русский рубеж язык конференция зарубежный конгресс проживать всемирный правительственный диаспора община координационный комиссия работа
Topic 4:
гражданин посольство консульский власть информация сотрудник ситуация виза находиться россиянин необходимый сми связь поступать погибший
Topic 5:
сша американский российский-американский вашингтон про обама клинтон госдепартамент вопрос госсекретарь администрация президент штат снв государственный
Topic 6:
снг содружество заседание глава совет государство-участник казах

In [163]:
get_topics(sorted_dfs[2].loc[:, 'text_clean'], 38, 15, ['nmf']) # 2013 - 2016

NMF
Topic 0:
международный развитие сотрудничество регион интерес работа экономический новый сегодня усилие мир безопасность проблема основа друг
Topic 1:
сирийский сирия оппозиция боевик террорист алеппо сара дамаск вооруженный группировка джабхат правительство женевский группа район
Topic 2:
украина украинский киев власть киевский юго-восток минский заявление ситуация донецкий донбасс действие донецк февраль договоренность
Topic 3:
визит двусторонний сотрудничество федерация переговоры встреча вопрос обсуждать ход актуальный состояться республика взаимодействие повестка торговый-экономический
Topic 4:
погибший выздоровление соболезнование пострадавший осуждать решительно террористический погибать выражать близкие скорый теракт родные ранение результат
Topic 5:
сша американский вашингтон администрация санкция против госдепартамент российский-американский керри про обама заявление белый санкционный госсекретарь
Topic 6:
оон резолюция сб совет безопасность генеральный право международны

In [220]:
get_topics(sorted_dfs[3].loc[:, 'text_clean'], 42, 15, ['nmf']) # С 2016

NMF
Topic 0:
друг сотрудничество сегодня уважаемый президент коллега рад важный международный регион интерес взаимодействие развитие контакт диалог
Topic 1:
сша американский вашингтон договор санкция американец военный вооружение администрация против госдепартамент заявление нато ракета дрсмд
Topic 2:
сирийский сирия турция астанинский процесс урегулирование оон резолюция иран сб конгресс оппозиция сочи астана встреча
Topic 3:
украина украинский киев киевский донбасс власть минский закон режим действие миссия порошенко нынешний право наблюдательный
Topic 4:
визит двусторонний сотрудничество переговоры федерация республика актуальный обсуждать региональный вопрос рабочий повестка ход мнение состояться
Topic 5:
мероприятие молодой участие дипломат федерация представитель принимать дипломатический состояться приглашать международный федеральный конференция посвящать зарубежный
Topic 6:
сирия сирийский террорист боевик провинция гуманитарный район зона житель лагерь пункт военный помощь ми

### Sorting data by unofficial dates

In [248]:
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs2 = sort_by_unofficial_dates(df)
get_stat(sorted_dfs2)

Corpus 0: 461 texts 197460 words
Corpus 1: 966 texts 571928 words
Corpus 2: 3849 texts 1372091 words
Corpus 3: 7096 texts 2318423 words


In [249]:
for df in sorted_dfs2:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [250]:
get_topics(sorted_dfs2[0].loc[:, 'text_clean'], 20, 15, ['nmf']) # До 2004

NMF
Topic 0:
сотрудничество международный организация развитие работа новый оон важный мир регион задача экономический вопрос интерес решение
Topic 1:
палестинский израиль дорожный карта пна четверка палестинец израильский посредник палестино-израильский ближневосточный урегулирование израильтянин возобновление администрация
Topic 2:
ирак оон иракский безопасность резолюция совет багдад временный суверенитет восстановление сб правительство генеральный гражданин урегулирование
Topic 3:
визит обсуждать вопрос переговоры двусторонний сотрудничество состояться ход президент встреча договариваться конкретный договоренность проблема республика
Topic 4:
ес евросоюз европейский европа пространство союз расширение общий безвизовый режим континент член единый россия-ес партнерство
Topic 5:
террористический осуждать соболезнование решительно акт жертва погибший взрыв акция выражать сочувствие пострадавший сообщение близкие результат
Topic 6:
терроризм борьба антитеррористический террорист террори

In [252]:
get_topics(sorted_dfs2[1].loc[:, 'text_clean'], 35, 15, ['nmf']) # 2004 - 2008

NMF
Topic 0:
сотрудничество вопрос обсуждать переговоры президент визит сфера взаимодействие сегодня договариваться встреча двусторонний договоренность развитие рамка
Topic 1:
политика мир международный мировой новый интерес война внешний становиться проблема глобальный угроза безопасность современный развитие
Topic 2:
израиль палестинец квартет пна палестинский израильский аббас палестино-израильский хамас дорожный карта посредник урегулирование территория ближневосточный
Topic 3:
европа совет се председательство европейский право конвенция комитет парламентский правовой ассамблея общеевропейский демократия защита разделительный
Topic 4:
развитие экономический регион сотрудничество область экономика организация региональный деятельность атр проект цель содействие система атэс
Topic 5:
палестинский администрация пна территория аббас национальный народ глава обстановка правительство хамас газа силовой политический урегулирование
Topic 6:
обсе организация смид безопасность государство-уч

In [253]:
get_topics(sorted_dfs2[2].loc[:, 'text_clean'], 38, 15, ['nmf']) # 2008 - 2014

NMF
Topic 0:
международный развитие сотрудничество новый работа мир интерес сегодня глобальный проблема важный основа мировой задача решение
Topic 1:
сирия сирийский оппозиция сара вооруженный дамаск женевский аннан насилие правительство прекращение лаг сириец боевик химический
Topic 2:
визит сотрудничество переговоры двусторонний федерация обсуждать республика вопрос состояться президент ход взаимодействие встреча рабочий актуальный
Topic 3:
осетия грузия южный абхазия грузинский тбилиси республика саакашвили закавказье август граница сила абхазский югоосетинский неприменение
Topic 4:
соотечественник рубеж конференция всемирный зарубежный конгресс проживать правительственный координационный федерация община диаспора комиссия работа русский
Topic 5:
оон резолюция сб совет безопасность генеральный международный решение организация ассамблея миссия сессия генассамблея генсекретарь секретарь
Topic 6:
гражданин сша американский посольство ребенок консульский суд власть информация сотрудник

In [254]:
get_topics(sorted_dfs2[3].loc[:, 'text_clean'], 65, 15, ['nmf']) # С 2014

NMF
Topic 0:
международный интерес мир проблема безопасность регион усилие решение работа развитие основа процесс принцип диалог новый
Topic 1:
сша американский вашингтон санкция администрация американец госдепартамент против санкционный российский-американский давление обама госсекретарь вводить военный
Topic 2:
сирия сирийский оппозиция урегулирование процесс сириец межсирийский женева правительство террорист сара политический переговоры астанинский группа
Topic 3:
украина украинский киев киевский власть юго-восток действие заявление нынешний язык верховный одесса очередной порошенко кризис
Topic 4:
визит актуальный переговоры двусторонний повестка рабочий вопрос обсуждать мнение региональный международный республика федерация обмен день
Topic 5:
война победа великий память мировой второй отечественный история летие нацизм ветеран исторический народ нацистский мероприятие
Topic 6:
химический озхо оружие применение запрещение химоружие вещество кзхо сирия конвенция отравлять муфс докл